In [26]:
cd /root/kubeflow-katib

/root/kubeflow-katib


In [2]:
! python setup.py install

In [3]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

In [4]:
import katib as kc
from katib import constants
from katib import utils
from katib import V1alpha3AlgorithmSetting
from katib import V1alpha3AlgorithmSetting
from katib import V1alpha3AlgorithmSpec
from katib import V1alpha3CollectorSpec
from katib import V1alpha3EarlyStoppingSetting
from katib import V1alpha3EarlyStoppingSpec
from katib import V1alpha3Experiment
from katib import V1alpha3ExperimentCondition
from katib import V1alpha3ExperimentList
from katib import V1alpha3ExperimentSpec
from katib import V1alpha3ExperimentStatus
from katib import V1alpha3FeasibleSpace
from katib import V1alpha3FileSystemPath
from katib import V1alpha3FilterSpec
from katib import V1alpha3GoTemplate
from katib import V1alpha3GraphConfig
from katib import V1alpha3Metric
from katib import V1alpha3MetricsCollectorSpec
from katib import V1alpha3NasConfig
from katib import V1alpha3ObjectiveSpec
from katib import V1alpha3Observation
from katib import V1alpha3Operation
from katib import V1alpha3OptimalTrial
from katib import V1alpha3ParameterAssignment
from katib import V1alpha3ParameterSpec
from katib import V1alpha3SourceSpec
from katib import V1alpha3Suggestion
from katib import V1alpha3SuggestionCondition
from katib import V1alpha3SuggestionList
from katib import V1alpha3SuggestionSpec
from katib import V1alpha3SuggestionStatus
from katib import V1alpha3TemplateSpec
from katib import V1alpha3Trial
from katib import V1alpha3TrialAssignment
from katib import V1alpha3TrialCondition
from katib import V1alpha3TrialList
from katib import V1alpha3TrialSpec
from katib import V1alpha3TrialStatus
from katib import V1alpha3TrialTemplate

In [15]:
algorithmsettings = V1alpha3AlgorithmSetting(
    name= "random_state",
    value = "10"
    )
algorithm = V1alpha3AlgorithmSpec(
    algorithm_name = "bayesianoptimization",
    algorithm_settings = [algorithmsettings]
  )

# Metric Collector
collector = V1alpha3CollectorSpec(kind = "TensorFlowEvent")
FileSystemPath = V1alpha3FileSystemPath(kind = "/train" , path = "Directory")
metrics_collector_spec = V1alpha3MetricsCollectorSpec(
    collector = collector,
    source = FileSystemPath)

# Objective
objective = V1alpha3ObjectiveSpec(
    goal = 0.9999,
    objective_metric_name = "Validation-accuracy",
    type = "maximize")

# Parameters

feasible_space_optimizer = V1alpha3FeasibleSpace(list = ["sgd", "adam", "ftrl"])
feasible_space_lr = V1alpha3FeasibleSpace(min = "0.01", max = "0.03")
feasible_space_layers = V1alpha3FeasibleSpace(min = "2", max = "5")

parameters = [V1alpha3ParameterSpec(
    feasible_space = feasible_space_optimizer, 
    name = "--optimizer",
    parameter_type = "categorical"
    ),
    V1alpha3ParameterSpec(
    feasible_space = feasible_space_lr, 
    name = "--lr",
    parameter_type ="double"
    ),
    V1alpha3ParameterSpec(
    feasible_space = feasible_space_layers, 
    name = "--num-layers",
    parameter_type = "int"
    )]

# Trialtemplate
go_template = V1alpha3GoTemplate(
    raw_template =   "apiVersion: \"batch/v1\"\nkind: Job\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n  template:\n    spec:\n      containers:\n      - name: {{.Trial}}\n        image: docker.io/kubeflowkatib/mxnet-mnist\n        command:\n        - \"python3\"\n        - \"/opt/mxnet-mnist/mnist.py\"\n        - \"--batch-size=64\"\n        {{- with .HyperParameters}}\n        {{- range .}}\n        - \"{{.Name}}={{.Value}}\"\n        {{- end}}\n        {{- end}}\n      restartPolicy: Never"
    )


trial_template= V1alpha3TrialTemplate(go_template=go_template)


# Experiment
experiment = V1alpha3Experiment(
    api_version="kubeflow.org/v1alpha3",
    kind="Experiment",
    metadata=V1ObjectMeta(name="bayesianoptimization-example",namespace="kube-prem"),

    spec=V1alpha3ExperimentSpec(
         algorithm = algorithm,
         max_failed_trial_count=3,
         max_trial_count=5,
         objective = objective,
         parallel_trial_count=5,
         parameters = parameters ,
         trial_template = trial_template
    )
)

In [22]:
namespace = kc.utils.get_default_target_namespace()
#experiment

In [17]:
kclient = kc.KatibClient()

## Create Experiment

In [29]:
kclient.create_experiment(experiment, namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-02-28T12:36:55Z',
  'generation': 1,
  'name': 'bayesianoptimization-example',
  'namespace': 'kube-prem',
  'resourceVersion': '3241401',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kube-prem/experiments/bayesianoptimization-example',
  'uid': '00d02b37-5a27-11ea-8d71-42010aa00012'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 5,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.9999,
   'objectiveMetricName': 'Validation-accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 5,
  'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
    'name': '--optimizer',
    'parameterType': 'categorical'},
   {'feasibleSpace': {'max': '0.03', 'min': '0.01'},
    'name': '--lr',
    'parameterTyp

## Get Single Experiment

In [20]:
kclient.get_experiment(name="bayesianoptimization-example", namespace=None)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-02-28T12:20:13Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 1,
  'name': 'bayesianoptimization-example',
  'namespace': 'kube-prem',
  'resourceVersion': '3232544',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kube-prem/experiments/bayesianoptimization-example',
  'uid': 'ab8ccc01-5a24-11ea-8d71-42010aa00012'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 5,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.9999,
   'objectiveMetricName': 'Validation-accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 5,
  'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
    'name': '--optimizer',
    'parameterType': 'categorical'},
   {'feasibleSpace': {'max': '0.03', 'min':

## Get all Experiment

In [21]:
kclient.get_experiment()


{'apiVersion': 'kubeflow.org/v1alpha3',
 'items': [{'apiVersion': 'kubeflow.org/v1alpha3',
   'kind': 'Experiment',
   'metadata': {'creationTimestamp': '2020-02-28T12:20:13Z',
    'finalizers': ['update-prometheus-metrics'],
    'generation': 1,
    'name': 'bayesianoptimization-example',
    'namespace': 'kube-prem',
    'resourceVersion': '3232573',
    'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kube-prem/experiments/bayesianoptimization-example',
    'uid': 'ab8ccc01-5a24-11ea-8d71-42010aa00012'},
   'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
     'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
    'maxFailedTrialCount': 3,
    'maxTrialCount': 5,
    'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
    'objective': {'goal': 0.9999,
     'objectiveMetricName': 'Validation-accuracy',
     'type': 'maximize'},
    'parallelTrialCount': 5,
    'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
      'name': '--o

## get experiment status

In [16]:
kclient.get_experiment_status(name="bayesianoptimization-example", namespace=namespace)


'Running'

## Is experiment succeeded

In [18]:
kclient.is_experiment_succeeded(name="bayesianoptimization-example", namespace=namespace)

False

## delete experiment

In [42]:
kclient.delete_experiment(name="bayesianoptimization2", namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-03-04T10:30:57Z',
  'deletionGracePeriodSeconds': 0,
  'deletionTimestamp': '2020-03-04T10:31:24Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 2,
  'name': 'bayesianoptimization2',
  'namespace': 'kube-prem',
  'resourceVersion': '7026286',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/kube-prem/experiments/bayesianoptimization2',
  'uid': '3be61c61-5e03-11ea-8d71-42010aa00012'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 20,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.9999,
   'objectiveMetricName': 'Validation-accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 7,
  'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
    'name': '--optimizer',
    'parame

## List Trials of an experiment

In [20]:
kclient.list_trials(name="bayesianoptimization-example", namespace=namespace)


[{'name': 'bayesianoptimization-example-q47w5qbw', 'status': 'Running'},
 {'name': 'bayesianoptimization-example-sxlzslkk', 'status': 'Running'},
 {'name': 'bayesianoptimization-example-w9m896g7', 'status': 'Running'},
 {'name': 'bayesianoptimization-example-xlzwp8p2', 'status': 'Running'},
 {'name': 'bayesianoptimization-example-z2nrxq5c', 'status': 'Running'}]

## List all experiments

In [21]:
kclient.list_experiments(namespace=namespace)


[{'name': 'bayesianoptimization-example', 'status': 'Running'},
 {'name': 'tfjob-example', 'status': 'Running'}]